In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

In [14]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('uint8')   
x_test = x_test.astype('uint8')

x_train.shape

x_train = x_train[:10000]
x_test = x_test[:10000]
y_train = y_train[:10000]
y_test = y_test[:10000]

In [15]:
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)
x_train = x_train.astype('float32')   
x_test = x_test.astype('float32')

In [16]:
kVals = [1,5,10]
accuracies = []
for k in kVals:
    # train the classifier with the current value of `k`
    model = KNeighborsClassifier(n_neighbors=k)

    model.fit(x_train, y_train)

    # evaluate the model and print the accuracies list
    score = model.score(x_test, y_test)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))
    accuracies.append(score)

k=1, accuracy=94.63%
k=5, accuracy=94.42%
k=10, accuracy=94.11%


In [17]:
kVals = [1,5,10]

for k in kVals:
    count = 0
    for i in range(len(x_test)):
        x = x_test[i]
        
        distances = [np.linalg.norm(x - data_point) for data_point in x_train]

        k_neighbors_indices = np.argsort(distances)[:k]
        k_neighbor_labels = [y_train[i] for i in k_neighbors_indices]
        
        # Return the most common class label
        most_common = np.bincount(k_neighbor_labels).argmax()
        if most_common == y_test[i]:
            count += 1
    score = count / len(x_test)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))



KeyboardInterrupt: 

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
# Train a Decision Tree on the data (Decision tree uses IG for the splits)
tree = DecisionTreeClassifier(random_state=0)
tree.fit(x_train, y_train)
importances = tree.feature_importances_
indices = np.argsort(importances)[::-1]
indices[0]

489

In [19]:
N = 100  # Change this to select number of features based on IG
selected_features = indices[:N]
print(selected_features)
# Apply the selection to training and test data
x_train_selected = x_train[:, selected_features]
x_test_selected = x_test[:, selected_features]

x_train[1][347]
# x_train_selected[1][0]

[489 155 350 435 239 432 656 486 347 539 522 381 290 351 295 570 300 296
 515 344 541 183 210 377 490 576 544 378 297 658 271 182 572  96 580 438
 354 345 516 318 407 213 625 157  99 488 220 319 437 431 190 654 353 436
 462 429 215 324 186 316 410 513 349 328 382 380 373 214 403 543 265 321
 371 400 568 440 266 527 602 269 246 545 519 657 293 404 100 498 291 329
 302 624 128 428 103 212 320 104 427 187]


71.0

In [20]:
kVals = [1, 5, 10, 20, 30]
accuracies = []

for k in kVals:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(x_train_selected, y_train)
    score = model.score(x_test_selected, y_test)
    print(f"k={k}, accuracy={score * 100:.2f}%")
    accuracies.append(score)

k=1, accuracy=93.48%
k=5, accuracy=93.45%
k=10, accuracy=92.54%
k=20, accuracy=91.55%
k=30, accuracy=90.94%


In [21]:
kVals = [5,10,20,30,50,100]
for k in kVals:
    count = 0
    for i in range(len(x_test_selected)):
        x = x_test_selected[i]

        distance_matrix = np.argsort(abs(x - x_train_selected), axis=0)
        k_neighbor_labels = np.array([])
        k_star = k
        for l in range(len(x)):
            if k_star <= 0: break
            # print(l)
            # print('...')
            # print(distance_matrix[:, l][:k_star])
            distances_along_axis = np.array(distance_matrix[:,l][:k_star])

            k_neighbor_labels = np.concatenate([k_neighbor_labels,distances_along_axis])
            k_star -= 1


        # k_neighbors_indices = np.argsort(distances)[:k]
        # k_neighbor_labels = [y_train[i] for i in k_neighbors_indices]
        
        # Return the most common class label
        # print(k_neighbor_labels)
        
        labels = y_train[np.array(k_neighbor_labels).flatten().astype(int)]

        most_common = np.bincount(labels).argmax()
        if most_common == y_test[i]:
            count += 1
    score = count / len(x_test_selected)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))

k=5, accuracy=25.71%


In [133]:
k_neighbor_labels

[array([  0, 338, 337, 336, 335], dtype=int64),
 array([249, 340, 338, 337], dtype=int64),
 array([  0, 288, 287], dtype=int64),
 array([  0, 247], dtype=int64)]

In [13]:
a = np.array([1,2,3,4])
a[np.array([1,2])]

array([2, 3])

In [88]:






print(np.argsort(abs(x_test_selected[0] - x_train_selected), axis=0))
abs(x_test_selected[0] - x_train_selected)


a = [
     [1.,4.,2.],
     [2.,0.,9.],
     [0.,9.,1.]
    ]
# np.argsort(a, axis=0)
print(np.argsort(a, axis=0))
np.argsort(a, axis=0)[:,1]
# a = [1,4,7,8,9]
# a[2:1]

[[249   0   0 ...  20 267   0]
 [340 288 247 ...  81 126 338]
 [338 287 419 ... 100 127 337]
 ...
 [ 66 413 393 ... 375  96 474]
 [376   9 415 ... 402 324 469]
 [143  37 452 ... 499 137  47]]
[[2 1 2]
 [0 0 0]
 [1 2 1]]


array([1, 0, 2], dtype=int64)

In [11]:
for k in kVals:
    # Calculate distances using vectorized operations
    distances = np.linalg.norm(x_test[:, np.newaxis, :] - x_train, axis=2)
    
    # Get indices of k-nearest training data points
    k_neighbors_indices = np.argsort(distances)[:, :k]
    
    # Get the labels of the k-nearest training data points
    k_neighbor_labels = y_train[k_neighbors_indices]
    
    # Return the most common class label along each row
    predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=k_neighbor_labels)

In [12]:
predictions

array([7, 2, 1, 0, 4, 1, 4, 9, 4, 7, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       1, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 1, 1, 1, 7, 4, 1,
       1, 5, 1, 1, 4, 4, 6, 3, 4, 1, 2, 0, 4, 1, 9, 1, 7, 8, 9, 9, 7, 9,
       1, 4, 3, 0, 7, 0, 2, 7, 1, 7, 3, 7, 9, 7, 9, 6, 2, 9, 1, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 1, 6, 9, 4, 0, 3, 4, 9, 9, 2, 1, 9, 4,
       8, 1, 1, 9, 7, 9, 9, 4, 9, 7, 3, 4, 7, 6, 9, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 9, 2, 0, 1, 9, 9, 4, 5,
       1, 5, 6, 0, 3, 1, 4, 6, 5, 4, 6, 5, 4, 5, 1, 9, 9, 7, 3, 3, 2, 1,
       1, 9, 1, 8, 1, 8, 1, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 1, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 1, 9, 3, 2, 9, 4, 9, 4, 1, 9, 0, 3, 5, 7, 5,
       7, 0, 2, 7, 1, 2, 8, 4, 1, 7, 5, 1, 8, 7, 9, 9, 2, 1, 4, 1, 5, 9,
       9, 7, 1, 1, 0, 6, 4, 1, 9, 1, 9, 5, 7, 7, 1, 1, 2, 0, 8, 1, 7, 7,
       9, 1, 0, 1, 5, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 1, 9, 7, 1, 9, 2,
       6, 4, 1, 5, 4, 3, 9, 2, 0, 9, 0, 0, 2, 8, 1,